<a href="https://colab.research.google.com/github/vibha-v-657/SegmindChallengeMyCode/blob/main/perfectsales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install h2o
import h2o
from h2o.automl import H2OAutoML
h2o.init(
    nthreads=2,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes

)


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpspt0woso
  JVM stdout: /tmp/tmpspt0woso/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpspt0woso/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,1 month and 1 day
H2O_cluster_name:,H2O_from_python_unknownUser_xg76d6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [ ]:
!unzip "/content/segmind_grand_ai_challenge_2021-dataset.zip"

Archive:  /content/segmind_grand_ai_challenge_2021-dataset.zip
replace dataset/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dataset/sample_submission.csv  
  inflating: dataset/TEST.csv        
  inflating: dataset/TRAIN.csv       
  inflating: dataset/store.csv       


In [ ]:
import pandas as pd
test_data = pd.read_csv('/content/dataset/TEST.csv')
train_data = pd.read_csv('/content/dataset/TRAIN.csv')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
store=pd.read_csv("/content/dataset/store.csv")

In [ ]:
holidays1=train_data['StateHoliday']
holidays2=test_data['StateHoliday']

train_data['StateHoliday_a']=(holidays1=='a')*1.0
train_data['StateHoliday_b']=(holidays1=='b')*1.0
train_data['StateHoliday_c']=(holidays1=='c')*1.0

test_data['StateHoliday_a']=(holidays2=='a')*1.0
test_data['StateHoliday_b']=(holidays2=='b')*1.0
test_data['StateHoliday_c']=(holidays2=='c')*1.0
# Splitting the values based on Date
s=pd.to_datetime(train_data['Date'],format='%Y-%m-%d')

train_data['year']=s.dt.year
train_data['month']=s.dt.month
train_data['day']=s.dt.day

train_data.pop('Date')

train_data.pop('Customers')

s=pd.to_datetime(test_data['Date'],format='%Y-%m-%d')

test_data['year']=s.dt.year
test_data['month']=s.dt.month
test_data['day']=s.dt.day

test_data.pop('Date')


0         2013-04-01
1         2013-03-16
2         2013-08-01
3         2013-05-23
4         2013-08-06
             ...    
305158    2013-05-28
305159    2013-03-02
305160    2013-09-16
305161    2013-06-22
305162    2013-09-07
Name: Date, Length: 305163, dtype: object

In [ ]:
#train_features.drop(['Date'],axis=1,inplace=True)
#test_features.drop(['Date'],axis=1,inplace=True)

train = pd.merge(train_data,store,on="Store")
test = pd.merge(test_data,store,on="Store")

In [ ]:
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
x = train.columns[:]
y = "Sales"
x.remove(y)
aml = H2OAutoML(max_models=1, seed=3)
aml.train(x=x, y=y, training_frame=train)
lb = aml.leaderboard
lb.head(rows=lb.nrows)

AutoML progress: |██

In [ ]:
perf = aml.predict(test)


In [ ]:
df= test.cbind(perf)


In [ ]:
res = df[:, ["predict"]]


In [ ]:
res.rename(columns={'predict':'Sales'})

In [ ]:
l=[]
for i in range(len(test)):
  l.append(i)

In [ ]:
h2o.export_file(res, path = "/content/output.csv", force = True)

In [ ]:
op=pd.read_csv("/content/output.csv")

In [ ]:
op.insert(0,"Index",l)

In [ ]:
op.to_csv('/content/output.csv',index=False)